<a href="https://colab.research.google.com/github/ihasdapie/Colab-Notebooks/blob/master/DataProcessingForSignatureVerification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title imports
from __future__ import division
import keras
from keras.preprocessing.image import img_to_array, load_img
import os
from matplotlib import pyplot as plt
import sklearn.metrics
import numpy as np
import cv2
from keras import Sequential
from keras.layers import Dense, Dropout, Input, Lambda, Flatten, Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import backend
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import BatchNormalization
from keras.activations import sigmoid
#import cupy as cp
from keras.regularizers import l2
# from keras.engine.topology import Layer
from keras.layers import Layer #or do i have to use the keras.engine.topolgy? will see.
from keras.utils import to_categorical, plot_model
from keras.models import Model
from keras.optimizers import Adam
import time
from keras.utils import Sequence
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# plt.xkcd()
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)


In [0]:
h, w = 500,300
shape = (h,w,1)
refdir = "/content/drive/My Drive/Brians_Signature_Verification_Project/Dataset/Genuine"
fakedir = "/content/drive/My Drive/Brians_Signature_Verification_Project/Dataset/Forgeries"

targetrefdir = "/content/drive/My Drive/Brians_Signature_Verification_Project/Dataset/ProcessedGenuine"
targetfakedir = "/content/drive/My Drive/Brians_Signature_Verification_Project/Dataset/ProcessedForgeries"

zipdir = "/content/drive/My Drive/Brians_Signature_Verification_Project/Dataset/ZipDataSet"


In [0]:
#@title save_img_to_dir(img_array,dir,label)
def save_img_to_dir(img_array, directory, label):
  for x in range(len(img_array)):
    cv2.imwrite(directory+"/"+label+str(x)+".png", img_array[x][:,:,0])

In [0]:
#@title load_data(dir, shape, label):
def load_data(dir, shape, label = "nolabel" ):

  img_array = [cv2.convertScaleAbs(img_to_array(load_img(dir+"/"+x, color_mode = "grayscale", target_size=(shape[0:2])))) for x in os.listdir(dir)]
  img_array = [img.reshape(shape) for img in img_array]
  img_array = np.asarray(img_array)
  if label == "nolabel":
    return img_array
  else:
    label_array = [label for x in range(len(img_array))]

    #use numpy arrays instead.

    # n_img = len(os.listdir(dir))

    # img_array = np.zeros((n_img, 1))

    # for x in os.listdir(dir):
    #   np.append(img_array, cv2.convertScaleAbs(img_to_array(load_img(dir+"/"+x, color_mode = "grayscale" ))))
    
    # label_array = np.zeros((n_img, 1))

    # for x in range(n_img):
    #   np.append(label_array, label)

    return img_array, label_array

In [0]:
#@title make_training_set(real,fake)
def make_training_set(real_dir, fake_dir, shape, size):
# return pairs & true/false labels

  X_real = load_data(real_dir, shape)/255
  X_fake = load_data(fake_dir, shape)/255
# 1 = genuine pair
# 0 = false pair

# list of size size of 0,1 for genuine/fake pairs
# 0 = real,real -> Y = 1 (true match)
# 1 = real,fake -> Y = 0 (no match)

#return_shape = (n_pairs, pairsize (2), h, w, 1)

  X_train = np.zeros(shape = (size, 2, shape[0], shape[1], 1))
  Y_train = np.zeros(shape = (size, 1))
  seed = np.random.randint(0, 2, size = size)
  for x in range(seed.shape[0]):
    if seed[x] == 0:
      img_1 = X_real[np.random.randint(0,X_real.shape[0])]
      img_2 = X_real[np.random.randint(0,X_real.shape[0])]
      X_train[x, 0, :, :, 0] = img_1.reshape(shape[0], shape[1])
      X_train[x, 1, :, :, 0] = img_2.reshape(shape[0], shape[1])
      Y_train[x] = 1

    if seed[x] == 1:
      img_1 = X_real[np.random.randint(0,X_real.shape[0])]
      img_2 = X_fake[np.random.randint(0,X_fake.shape[0])]
      X_train[x, 0, :, :, 0] = img_1.reshape(shape[0], shape[1])
      X_train[x, 1, :, :, 0] = img_2.reshape(shape[0], shape[1])
      Y_train[x] = 0

  # Y_train = to_categorical(Y_train) #techically not necessary because everything is 0/1 already, but I wanted to do it.

  return X_train, Y_train

In [0]:
#@title show_img
def show_img(img, title= ""):
  plt.imshow(img, cmap = 'gray')
  plt.title(title)

In [0]:
#@title pre_process_data(dir, h, w):
def pre_process_data(dir, h, w):
#returns X_array w/ superimposed centroids & bounding boxes, labels

#load images in grayscale, resize, turn to array

  grayscale_img_array = [(img_to_array(load_img(dir+"/"+x, color_mode = "grayscale", target_size = (h,w)))) for x in os.listdir(dir)]

#try using openCV imread

  # grayscale_img_array = [cv2.imread(img, cv2.IMREAD_GRAYSCALE) for img in os.listdir(dir)]
  # cv2_imshow(grayscale_img_array[1])

  grayscale_img_array=[(cv2.convertScaleAbs(img)) for img in grayscale_img_array] #convert to uint8 format
#Thresholded images are used to find the weighted centroids of each image. 
#Thresholding is unnecessary but it was a learning experience. 

  thresholded_img_array = [cv2.threshold(img, 170,255,cv2.THRESH_BINARY_INV) for img in grayscale_img_array] #threshold binary is set higher that 127 due to image faintness
  thresholded_img_array = [thresh[1] for thresh in thresholded_img_array] #threshold returns (ret, thresh). Only thresh is needed.

  img_moments = [cv2.moments(thresh) for thresh in thresholded_img_array] #"center of mass" of image. This can be used to find the centroid of the image. 

  center_x_pos = [int(m["m10"] / m["m00"]) for m in img_moments]
  center_y_pos = [int(m["m01"] / m["m00"]) for m in img_moments]

# add "crosshair" to center of grayscale images
# is there a way to do this in one list comprehension hm

  for x in range(len(grayscale_img_array)):
    (grayscale_img_array[x])[:, center_x_pos[x]] = 0
    (grayscale_img_array[x])[center_y_pos[x], :] = 0

#add bounding box to grayscale images
 
  img_contour_array = [cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) for thresh in thresholded_img_array]


  for i in range(len(grayscale_img_array)):
    x,y,w,h= cv2.boundingRect(img_contour_array[i][0])
    grayscale_img_array[i] = cv2.rectangle(grayscale_img_array[i],(x,y),(x+w,y+h),(0,255,0),1)
#get filenames
  labels = [x.split(".")[0] for x in os.listdir(dir)]
  return grayscale_img_array, labels


In [0]:
#Cannot train at high enough resolution to justify the data-preprocessing done, so will simply load it & turn into grayscale
ref = load_data(refdir, shape)
fake = load_data(fakedir, shape)


In [0]:
ref[:,:,:,0].shape

(48, 500, 300)

In [0]:
#save to dir
save_img_to_dir( fake, targetfakedir, "Fake_")
save_img_to_dir(ref, targetrefdir,  "Real_")

In [0]:
#access .zip files
pip install gdown

In [0]:
!gdown --id 1v5eGVp6aaSIsozIgmB1oOQjQU-l0AH90 --output /content/data.zip

Downloading...
From: https://drive.google.com/uc?id=1v5eGVp6aaSIsozIgmB1oOQjQU-l0AH90
To: /content/data.zip
3.99MB [00:00, 128MB/s]


In [0]:
import zipfile
z = zipfile.ZipFile('/content/data.zip', 'r')
z.extractall('/content/dataset')
z.close()